In [17]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [18]:

data = """Vamsi Teja: A journey of resilience and dreams,
From humble beginnings, his ambition gleams.
Through challenges faced, he never strays,
Each setback fuels his unwavering blaze."""

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
vocab_size = len(tokenizer.word_index) + 1

In [20]:
sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

In [21]:
max_sequence_len = max([len(seq) for seq in sequences])
sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

In [22]:

X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [23]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 3.2586 - accuracy: 0.0455
Epoch 2/500
1/1 [==============================] - 0s 17ms/step - loss: 3.2565 - accuracy: 0.0909
Epoch 3/500
1/1 [==============================] - 0s 16ms/step - loss: 3.2544 - accuracy: 0.0909
Epoch 4/500
1/1 [==============================] - 0s 15ms/step - loss: 3.2522 - accuracy: 0.0455
Epoch 5/500
1/1 [==============================] - 0s 17ms/step - loss: 3.2500 - accuracy: 0.0909
Epoch 6/500
1/1 [==============================] - 0s 15ms/step - loss: 3.2477 - accuracy: 0.0909
Epoch 7/500
1/1 [==============================] - 0s 14ms/step - loss: 3.2453 - accuracy: 0.0909
Epoch 8/500
1/1 [==============================] - 0s 17ms/step - loss: 3.2426 - accuracy: 0.0909
Epoch 9/500
1/1 [==============================] - 0s 15ms/step - loss: 3.2398 - accuracy: 0.0909
Epoch 10/500
1/1 [==============================] - 0s 15ms/step - loss: 3.2367 - accuracy: 0.0909
Epoch 11/500
1/1 [===

In [25]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

generated_text = generate_text("The", 10, model, max_sequence_len)
print(generated_text)

The humble humble beginnings his ambition gleams blaze dreams blaze blaze
